# Main Code

In [1]:
!pip install bert-score
#!pip install git+https://github.com/neulab/BARTScore.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [2]:
!git clone https://github.com/neulab/BARTScore.git


Cloning into 'BARTScore'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 220 (delta 18), reused 14 (delta 14), pack-reused 194 (from 1)
Receiving objects: 100% (220/220), 101.98 MiB | 23.05 MiB/s, done.
Resolving deltas: 100% (47/47), done.
Updating files: 100% (192/192), done.


In [3]:
import sys
sys.path.append("./BARTScore")
# Now import
from bart_score import BARTScorer

2025-05-13 10:47:21.006947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747133241.208396      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747133241.263944      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
"""Imports go here"""

from transformers import ViltProcessor, ViltForQuestionAnswering
import torch
from PIL import Image
import requests

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
#from word2number import w2n


In [7]:
""" Load CSV and split the data """

# Load the CSV file
csv_path = "/kaggle/input/qna-final/qna_final.csv"
df = pd.read_csv(csv_path)

# Get unique Item_IDs
unique_ids = df["Item_ID"].unique()

# Set random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)

# Shuffle and split the unique IDs
train_ids, temp_ids = train_test_split(unique_ids, test_size=0.3, random_state=random_seed)  # 70% train
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=random_seed)      # 15% val, 15% test

# Create train, val, and test DataFrames
train_df = df[df["Item_ID"].isin(train_ids)]
val_df = df[df["Item_ID"].isin(val_ids)]
test_df = df[df["Item_ID"].isin(test_ids)]

print(f"Train size: {len(train_df)}, Val size: {len(val_df)}, Test size: {len(test_df)}")


Train size: 16666, Val size: 3663, Test size: 3558


In [8]:
"""Filtering to only retain answers in label2id"""
# Load the model and processor to access label2id
model_config_source = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
known_answers = set(model_config_source.config.label2id.keys())

# Your text_to_num mapping logic (ensure it's accessible here)
# This is a simplified version of what's in your QnADataset
def get_text_to_num_mapping():
    text_to_num = {
        "zero": "0", "one": "1", "two": "2", "three": "3", "four": "4",
        "five": "5", "six": "6", "seven": "7", "eight": "8", "nine": "9",
        "ten": "10", "eleven": "11", "twelve": "12", "thirteen": "13",
        "fourteen": "14", "fifteen": "15", "sixteen": "16", "seventeen": "17",
        "eighteen": "18", "nineteen": "19", "twenty": "20",
    }
    for i in range(21, 1001): # Assuming your mapping goes up to 1000
        text_to_num[str(i)] = str(i)
    return text_to_num

text_to_num_map = get_text_to_num_mapping()

def normalize_answer(answer_str):
    normalized = str(answer_str).strip().lower()
    return text_to_num_map.get(normalized, normalized)

# --- Apply filtering to your DataFrames ---
# Example for one DataFrame, repeat for train_df, val_df, test_df

original_test_df_len = len(test_df)
test_df['normalized_answer'] = test_df['Answer'].apply(normalize_answer)
test_df_filtered = test_df[test_df['normalized_answer'].isin(known_answers)]
filtered_test_df_len = len(test_df_filtered)

print(f"Original test_df size: {original_test_df_len}")
print(f"Filtered test_df size: {filtered_test_df_len} ({ (filtered_test_df_len / original_test_df_len) * 100 :.2f}% retained)")

# Now, use test_df_filtered (and similarly filtered train/val DataFrames) to create your datasets:
# test_dataset = QnADataset(test_df_filtered, image_dir, processor, model_config_source.config.label2id)

# In your QnADataset.__getitem__, you would still use the normalized answer
# but the check `if processed_answer_str in self.label2id:` should now always pass
# if you use the filtered DataFrame. You might keep the error/warning for safety,
# but it ideally shouldn't be triggered.

Original test_df size: 3558
Filtered test_df size: 2909 (81.76% retained)


/tmp/ipykernel_31/1814277403.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['normalized_answer'] = test_df['Answer'].apply(normalize_answer)


In [10]:
"""Create a custom dataset class"""

#text_form = w2n.to_words(i)  # Convert integer to text form

class QnADataset(Dataset):
    def __init__(self, dataframe, image_dir, processor, label2id): # Processor is not strictly needed here anymore, but label2id is
        self.dataframe = dataframe
        self.image_dir = image_dir
        # self.processor = processor # Not used directly in __getitem__ anymore
        self.label2id = label2id
        self.text_to_num = self.generate_text_to_num_mapping()

    def generate_text_to_num_mapping(self):
        # (Your existing generate_text_to_num_mapping method - keep as is)
        text_to_num = {
            "zero": "0", "one": "1", "two": "2", "three": "3", "four": "4",
            "five": "5", "six": "6", "seven": "7", "eight": "8", "nine": "9",
            "ten": "10", "eleven": "11", "twelve": "12", "thirteen": "13",
            "fourteen": "14", "fifteen": "15", "sixteen": "16", "seventeen": "17",
            "eighteen": "18", "nineteen": "19", "twenty": "20",
        }
        for i in range(21, 1001):
            text_to_num[str(i)] = str(i)
        return text_to_num

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = f"{self.image_dir}/{row['Image_Path']}"
        question_text = row["Question"]  # Keep as raw text
        answer_str = row["Answer"].strip().lower()

        # Convert text-based numbers to numerical strings if needed
        if answer_str in self.text_to_num:
            processed_answer_str = self.text_to_num[answer_str]
        else:
            processed_answer_str = answer_str

        # Load PIL image
        try:
            pil_image = Image.open(image_path).convert("RGB")
        except FileNotFoundError:
            print(f"Error: Image not found at {image_path}")
            # Handle appropriately: skip, return None, or use a placeholder
            # For now, let's re-raise to make it obvious during debugging
            raise
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            raise


        # Encode the answer string to an ID
        if processed_answer_str in self.label2id:
            answer_id = self.label2id[processed_answer_str]
        else:
            # This can be a common issue. Ensure your label2id covers all possible answers
            # or you have a strategy for unknown answers.
            print(f"Warning: Answer '{processed_answer_str}' (original: '{row['Answer']}') not found in label2id mapping. Item index: {idx}, Image: {row['Image_Path']}")
            # Option 1: Raise error (as you had)
            # raise ValueError(f"Answer '{processed_answer_str}' not found in label2id mapping.")
            # Option 2: Assign a default/unknown ID if you have one, or skip sample.
            # For now, let's make it return something that might cause a downstream error if not handled,
            # or you can choose to filter these out in collate_fn or raise error.
            # To proceed, we'll assume for now this case should be an error or filtered.
            # For robust training, you'd need a clear strategy.
            raise ValueError(f"Answer '{processed_answer_str}' (from original '{row['Answer']}') not found in label2id mapping for image {row['Image_Path']}.")


        return {
            "image": pil_image,          # Return the PIL Image object
            "question": question_text,   # Return the raw question string
            "labels": torch.tensor(answer_id, dtype=torch.long) # Return the label as a tensor
        }

In [11]:
""" Custom collate fn """
def collate_fn(batch_list_of_dicts, processor_instance, num_possible_answers): # Added num_possible_answers
    """
    Custom collate function to handle batching of images and questions for ViLT.
    batch_list_of_dicts: A list of dictionaries, each from QnADataset.__getitem__
    processor_instance: The ViltProcessor instance
    num_possible_answers: The total number of labels the model can predict (e.g., model.config.num_labels)
    """
    images = [item["image"] for item in batch_list_of_dicts]
    questions = [item["question"] for item in batch_list_of_dicts]

    inputs = processor_instance(
        images=images,
        text=questions,
        return_tensors="pt",
        padding="longest",
        truncation=True,
    )

    # Extract answer IDs (which are scalar tensors from __getitem__)
    answer_id_tensors = [item["labels"] for item in batch_list_of_dicts]

    # Create the target labels tensor for BCEWithLogitsLoss
    # Shape: (batch_size, num_possible_answers)
    # Initialize with zeros, dtype float32 as BCEWithLogitsLoss expects float targets.
    batched_labels = torch.zeros(len(batch_list_of_dicts), num_possible_answers, dtype=torch.float)

    for i, single_answer_id_tensor in enumerate(answer_id_tensors):
        answer_id = single_answer_id_tensor.item()  # Convert scalar tensor (e.g., tensor(5)) to Python int (e.g., 5)

        # Ensure the answer_id is within the valid range
        if 0 <= answer_id < num_possible_answers:
            batched_labels[i, answer_id] = 1.0  # Set the target to 1.0 for the correct answer_id
        else:
            # This is a critical issue if it happens. It means your label2id mapping in the Dataset
            # produced an ID that is out of bounds for the model's output layer.
            # Ensure model.config.label2id was used to generate these IDs.
            print(f"CRITICAL WARNING: Invalid answer_id {answer_id} encountered for num_possible_answers {num_possible_answers}.")
            print(f"This usually means an inconsistency between your dataset's answer encoding and the model's vocabulary.")
            # You might want to raise an error here or handle it based on your needs.
            # For now, this label will remain all zeros, which is incorrect.
            # raise ValueError(f"Invalid answer_id {answer_id} for num_labels {num_possible_answers}")


    inputs["labels"] = batched_labels
    return inputs

In [12]:
""" Prepare dataloaders """
# Load the model (you likely already have this when preparing datasets or evaluation)
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa") # Ensure processor is also loaded

# Get num_labels from the loaded model
num_labels_for_model = model.config.num_labels # This will be 3129 for "dandelin/vilt-b32-finetuned-vqa"

# Directory containing the images
image_dir = "/kaggle/input/filtered-small-amazon-qna" # Ensure this path is correct

# Create datasets (ensure you are using model.config.label2id here)
train_dataset = QnADataset(train_df, image_dir, processor, model.config.label2id)
val_dataset = QnADataset(val_df, image_dir, processor, model.config.label2id)
test_dataset = QnADataset(test_df_filtered, image_dir, processor, model.config.label2id)

# Create DataLoaders, passing num_labels_for_model to the collate_fn via the lambda
train_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=lambda batch: collate_fn(batch, processor, num_labels_for_model)
)
val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=lambda batch: collate_fn(batch, processor, num_labels_for_model)
)
test_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=lambda batch: collate_fn(batch, processor, num_labels_for_model)
)

In [13]:
import sys
import time
import torch
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 1. Add local BARTScore code into Python’s import path
sys.path.append("./BARTScore")

# 2. Semantic‐similarity imports
from bert_score import score as bert_score
from bart_score import BARTScorer

# 3. Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 4. Move your model to device & set eval mode
model.to(device)
model.eval()

# 5. Accumulators
all_pred_ids   = []
all_true_ids   = []
all_pred_texts = []
all_true_texts = []

# --- Start overall timer ---
t0_overall = time.time()

# 6. Inference + gather labels/texts with progress bar
t0_loop = time.time()
for batch_idx, batch in enumerate(tqdm(test_loader, desc="Evaluating batches")):
    # a) Send batch to device
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    logits  = outputs.logits

    # b) Get predicted & true IDs
    pred_ids = logits.argmax(dim=-1)
    true_ids = batch['labels'].argmax(dim=-1)

    # c) Flatten & gather for metrics
    pred_flat = pred_ids.view(-1).cpu().numpy()
    true_flat = true_ids.view(-1).cpu().numpy()
    all_pred_ids.extend(pred_flat)
    all_true_ids.extend(true_flat)

    # d) Convert IDs → label strings
    all_pred_texts.extend([model.config.id2label[i] for i in pred_flat])
    all_true_texts.extend([model.config.id2label[i] for i in true_flat])
t1_loop = time.time()
print(f"\nInference & data gathering took {t1_loop - t0_loop:.2f} sec")

# 7. Compute classification metrics
t0_cls = time.time()
accuracy  = accuracy_score(all_true_ids, all_pred_ids)
precision = precision_score(all_true_ids, all_pred_ids, average='macro', zero_division=0)
recall    = recall_score(all_true_ids, all_pred_ids, average='macro', zero_division=0)
f1        = f1_score(all_true_ids, all_pred_ids, average='macro', zero_division=0)
t1_cls = time.time()
print(f"Classification metrics computed in {t1_cls - t0_cls:.2f} sec")

print("\n=== Classification Metrics ===")
print(f"Accuracy      : {accuracy:.4f}")
print(f"Precision (M) : {precision:.4f}")
print(f"Recall    (M) : {recall:.4f}")
print(f"F1 Score  (M) : {f1:.4f}")

# 8. BERTScore (semantic similarity)
t0_bert = time.time()
bert_p, bert_r, bert_f1 = bert_score(
    all_pred_texts,
    all_true_texts,
    lang='en',
    model_type='bert-base-uncased',
    rescale_with_baseline=True
)
t1_bert = time.time()
print(f"\nBERTScore computed in {t1_bert - t0_bert:.2f} sec")

print("\n=== BERTScore ===")
print(f"Precision : {bert_p.mean().item():.4f}")
print(f"Recall    : {bert_r.mean().item():.4f}")
print(f"F1        : {bert_f1.mean().item():.4f}")

# 9. BARTScore (semantic entailment)
t0_bart = time.time()
bart_scorer = BARTScorer(device=device.type, checkpoint='facebook/bart-large-cnn')
bart_scores = bart_scorer.score(
    all_pred_texts,
    all_true_texts,
    batch_size=8
)
t1_bart = time.time()
print(f"\nBARTScore computed in {t1_bart - t0_bart:.2f} sec")

print("\n=== BARTScore ===")
mean_bart = sum(bart_scores) / len(bart_scores)
print(f"Mean score: {mean_bart:.4f}")


# --- End overall timer ---
t1_overall = time.time()
print(f"\nTotal script time: {t1_overall - t0_overall:.2f} sec")


Using device: cuda


Evaluating batches: 100%|██████████| 364/364 [01:18<00:00,  4.65it/s]



Inference & data gathering took 78.26 sec
Classification metrics computed in 0.02 sec

=== Classification Metrics ===
Accuracy      : 0.5548
Precision (M) : 0.1008
Recall    (M) : 0.1074
F1 Score  (M) : 0.0895


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]


BARTScore computed in 15.64 sec

=== BARTScore ===
Mean score: -4.0996

Total script time: 100.53 sec


In [20]:
""" Use test loader for baseline evaluation """

# Assume 'model', 'test_loader', and 'device' are already defined and configured correctly.
# model.config.id2label should also be available.

# Initialize accumulators for metrics
total_correct_predictions = 0
total_samples_processed = 0

model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculations for inference
    for batch_idx, batch in enumerate(test_loader):
        # Move batch data to the configured device (GPU or CPU)
        # The collate_fn ensures 'labels' is correctly shaped [batch_size, num_classes]
        input_batch = {k: v.to(device) for k, v in batch.items()}

        # Perform forward pass
        outputs = model(**input_batch)
        logits = outputs.logits  # Raw scores from the model, shape [batch_size, num_classes]

        # Get predicted answer IDs by finding the index with the highest logit
        predicted_ids = logits.argmax(dim=-1)  # Shape: [batch_size]

        # Get ground truth answer IDs from the multi-hot encoded labels
        # batch['labels'] has shape [batch_size, num_classes] with 1.0 at the correct answer index
        true_label_ids = input_batch['labels'].argmax(dim=-1) # Shape: [batch_size]

        # Compare predicted IDs with true IDs to find correct predictions in this batch
        correct_predictions_in_batch = (predicted_ids == true_label_ids).sum().item()
        
        # Accumulate total correct predictions and total samples
        total_correct_predictions += correct_predictions_in_batch
        total_samples_processed += true_label_ids.size(0) # batch_size

        # --- Optional: Print predictions and ground truths for this batch for inspection ---
        predicted_answers_text = [model.config.id2label.get(idx.item(), "ID_NOT_FOUND") for idx in predicted_ids]
        true_answers_text = [model.config.id2label.get(idx.item(), "ID_NOT_FOUND") for idx in true_label_ids]

        print(f"--- Batch {batch_idx + 1} ---")
        for i in range(len(predicted_answers_text)):
            print(f"Sample {i}: Predicted: '{predicted_answers_text[i]}' (ID: {predicted_ids[i].item()}), "
                  f"Ground Truth: '{true_answers_text[i]}' (ID: {true_label_ids[i].item()}), "
                  f"Correct: {predicted_ids[i] == true_label_ids[i]}")
        print(f"Correct in this batch: {correct_predictions_in_batch}/{true_label_ids.size(0)}")
        print("-" * 30)
        # --- End Optional Printing ---

# Calculate overall accuracy
if total_samples_processed > 0:
    accuracy = (total_correct_predictions / total_samples_processed) * 100
    print(f"\n--- Overall Evaluation Metrics ---")
    print(f"Total Samples Processed: {total_samples_processed}")
    print(f"Total Correct Predictions: {total_correct_predictions}")
    print(f"Accuracy: {accuracy:.2f}%")
else:
    print("\nNo samples were processed. Cannot calculate accuracy.")
    if not test_loader.dataset: # Check if the dataset itself is empty
        print("The test dataset appears to be empty.")
    else:
        print("Consider if all samples were filtered out or if the dataloader is not yielding data.")

--- Batch 1 ---
Sample 0: Predicted: 'no' (ID: 9), Ground Truth: 'no' (ID: 9), Correct: True
Sample 1: Predicted: 'no' (ID: 9), Ground Truth: 'no' (ID: 9), Correct: True
Sample 2: Predicted: 'no' (ID: 9), Ground Truth: 'no' (ID: 9), Correct: True
Sample 3: Predicted: '4' (ID: 53), Ground Truth: '2' (ID: 17), Correct: False
Sample 4: Predicted: 'orange' (ID: 2), Ground Truth: 'blue' (ID: 13), Correct: False
Sample 5: Predicted: 'orange' (ID: 2), Ground Truth: 'blue' (ID: 13), Correct: False
Sample 6: Predicted: 'orange' (ID: 2), Ground Truth: 'blue' (ID: 13), Correct: False
Sample 7: Predicted: 'orange' (ID: 2), Ground Truth: 'blue' (ID: 13), Correct: False
Correct in this batch: 3/8
------------------------------
--- Batch 2 ---
Sample 0: Predicted: 'yes' (ID: 3), Ground Truth: 'yes' (ID: 3), Correct: True
Sample 1: Predicted: 'yes' (ID: 3), Ground Truth: 'yes' (ID: 3), Correct: True
Sample 2: Predicted: 'yes' (ID: 3), Ground Truth: 'yes' (ID: 3), Correct: True
Sample 3: Predicted: 'ye

In [30]:
label2id = model.config.label2id

if '2' in label2id:
    print(f"'2' exists in label2id with ID: {label2id['2']}")
else:
    print("'2' does not exist in label2id")

'2' exists in label2id with ID: 17


In [2]:
"""ViLT pipeline"""


# Load the processor and model
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

# Set the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Replace with your image URL and question
image_url = "https://img1.hscicdn.com/image/upload/f_auto,t_ds_w_720/lsci/db/PICTURES/CMS/188300/188325.jpg"
question = "What is on the man's head"

# Load and preprocess the image
image = Image.open(requests.get(image_url, stream=True).raw).convert("RGB")
inputs = processor(images=image, text=question, return_tensors="pt").to(device)

# Perform forward pass
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted answer
logits = outputs.logits
predicted_id = logits.argmax(-1).item()
answer = model.config.id2label[predicted_id]

print(f"Predicted Answer: {answer}")

NameError: name 'ViltProcessor' is not defined